# U1. Imports
- This imports Python packages, display settings, and some common variable and stat lists
- Type: Utility
- Run Frequency: Frequent
- Created: 11/1/2023
- Updated: 8/20/2025

### Packages

In [2]:
import ast
import concurrent.futures
import csv
import cv2
import datetime
import dateutil.parser
import distutils.dir_util
import gc
import glob
import IPython.display
import ipywidgets as widgets
import joblib
import json
import keras
import math
import matplotlib.pyplot as plt
import numpy as np
import openmeteo_requests
import os
import pandas as pd
import pathlib
import pickle
import polars as pl
import plotly.express as px
import pyautogui
import pyperclip
import pytz
import re
import requests
import requests_cache
import seaborn as sns
import selenium
import shutil
import smtplib
import ssl
import sqlite3
import statsapi
import statsmodels.formula.api as smf
import statsmodels.api as sm
import subprocess
import sys
import tensorflow as tf
import time
import traceback
import unidecode
import warnings
import webbrowser
import xlrd
import random
import urllib
import zipfile

from bs4 import BeautifulSoup
from collections import Counter
from concurrent.futures import ThreadPoolExecutor
from copy import deepcopy
from dateutil import parser
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from functools import partial
from io import StringIO
from IPython.display import display, Javascript, clear_output
from joblib import Parallel, delayed
from lxml import html
from openpyxl import load_workbook
from paretoset import paretoset
from pathlib import Path
from pulp import GLPK_CMD  
from pybaseball import statcast
from pydfs_lineup_optimizer import get_optimizer, Site, Sport, Player, TeamStack, PlayerFilter, RandomFantasyPointsStrategy, ProgressiveFantasyPointsStrategy, AfterEachExposureStrategy, LineupOptimizer
# from pydfs_lineup_optimizer.solvers.mip_solver import MIPSolver
from pydfs_lineup_optimizer.solvers import PuLPSolver
from retry_requests import retry
from scipy import stats
from scipy.stats import ttest_ind, mannwhitneyu, ttest_1samp
from scipy.stats.mstats import winsorize
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.svm import SVC
from statsapi import get
from sqlalchemy import create_engine
from tensorflow.keras.losses import KLDivergence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm
from urllib.request import urlopen, Request
from webdriver_manager.chrome import ChromeDriverManager

### Display Options

In [ ]:
warnings.simplefilter(action="ignore")

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.float_format', '{:.6f}'.format)

### Paths

In [ ]:
model_path = r"C:\Users\james\Documents\MLB\Models"
baseball_path = r"C:\Users\james\Documents\MLB\Database"
download_path = r"C:\Users\james\Downloads"

### Dates

In [ ]:
# Today's Date
# YYYY-MM-DD (datetime)
todaysdate_dt = datetime.date.today()

# YYYY-MM-DD (string)
todaysdate_dash = str(todaysdate_dt)

# MM/DD/YYYY
todaysdate_slash = todaysdate_dash.split("-")
todaysdate_slash = todaysdate_slash[1] + "/" + todaysdate_slash[2] + "/" + todaysdate_slash[0]

# YYYYMMDD
todaysdate = todaysdate_dash.replace("-", "")

## MM-DD-YYYY
todaysdate_dash = todaysdate[:4] + "-" + todaysdate[4:6] + "-" + todaysdate[6:]

In [ ]:
# Get the current date
current_date = datetime.datetime.now()

# Subtract one day from the current date to get yesterday's date
yesterday_dt = current_date - datetime.timedelta(days=1)

# Format yesterday's date as "YYYYMMDD"
yesterdaysdate = yesterday_dt.strftime("%Y%m%d")

# MM/DD/YYYY
yesterdaysdate_slash = yesterdaysdate[4:6] + "/" + yesterdaysdate[6:8] + "/" + yesterdaysdate[0:4] 

## MM-DD-YYYY
yesterdaysdate_dash = yesterdaysdate[:4] + "-" + yesterdaysdate[4:6] + "-" + yesterdaysdate[6:]

### Stat Lists

In [2]:
# Plate apperance events 
events_list = ['b1','b2','b3','hr','bb','hbp','so','fo','go','lo','po']
events_list = ['b1','b2','b3','bb','fo','go','hbp','hr','lo','po','so']

# Adjusted for park
events_list_adj = [f'{event}_adj' for event in events_list]

# List of statcast stats
statcast_list = ['estimated_woba_using_speedangle','to_left','to_middle','to_right','hard_hit','barrel']
# Stats for which we need to calculate rates
calc_list = ['iso','slg','obp','woba']
# Stats for which we want maximums over time
max_list = ['totalDistance','maxSpeed','maxSpin','launchSpeed']

# Stats for which we want averages over time
avg_list = events_list + statcast_list 

# All stats
stat_list = avg_list + calc_list + max_list + ['ab','pa']

### Batter
# Create position/length-specific inputs from stats
batter_avg_short = [f"{stat}_b" for stat in avg_list]
batter_avg_long =  [f"{stat}_b_long" for stat in avg_list]

batter_calc_short = [f"{stat}_b" for stat in calc_list]
batter_calc_long = [f"{stat}_b_long" for stat in calc_list]

batter_max_short = [f"{stat}_b" for stat in max_list]
batter_max_long =  [f"{stat}_b_long" for stat in max_list]

batter_stats_short = [f"{stat}_b" for stat in stat_list]
batter_stats_long  = [f"{stat}_b_long" for stat in stat_list]

# Steamer stats
batter_stats_fg =     ['b1_rate','b2_rate','b3_rate','hr_rate','bb_rate','hbp_rate','so_rate','woba','slg','obp']
# For use in imputations
batter_stats_fg_imp = batter_stats_fg + ['b_L', 'p_L']


### Pitcher
# Create position/length-specific inputs from stats
pitcher_avg_short = [f"{stat}_p" for stat in avg_list]
pitcher_avg_long =  [f"{stat}_p_long" for stat in avg_list]

pitcher_calc_short = [f"{stat}_p" for stat in calc_list]
pitcher_calc_long = [f"{stat}_p_long" for stat in calc_list]

pitcher_max_short = [f"{stat}_p" for stat in max_list]
pitcher_max_long =  [f"{stat}_p_long" for stat in max_list]

pitcher_stats_short = [f"{stat}_p" for stat in stat_list]
pitcher_stats_long  = [f"{stat}_p_long" for stat in stat_list]

# Steamer stats (for imputing)
pitcher_stats_fg =    ['H9','HR9','K9','BB9','GBrate','FBrate','LDrate','SIERA']
# Add stats for projecting pulls
pitcher_stats_fg2 = pitcher_stats_fg + ['reliability','IP_start','IP','relief_IP']
# For use in imputations
pitcher_stats_fg_imp = pitcher_stats_fg + ['b_L', 'p_L']

['b1_adj', 'b2_adj', 'b3_adj', 'hr_adj', 'bb_adj', 'hbp_adj', 'so_adj', 'fo_adj', 'go_adj', 'lo_adj', 'po_adj']


In [ ]:
# Game state
game_info_list = ['date', 'gamePk', 'atBatIndex', 'inning', 'halfInning', 'outs', 'awayscore', 'homescore']

# Player info
batter_info_list = ['batterName', 'batter', 'batSide']
pitcher_info_list = ['pitcherName', 'pitcher', 'pitchHand']
           
# PAs and ABs
pa_list = ['pa_b', 'ab_b', 'pa_p', 'ab_p', 'pa_b_long', 'ab_b_long', 'pa_p_long', 'ab_p_long']
    
# Base running
baserunning_stats = ["sba_2b", "sb_2b", "sba_3b", "sb_3b"]

### PA Model

In [ ]:
### Exclusions
# Stats that do not apply to the position or we just don't want
pa_exclude = ["maxSpeed_b", "maxSpin_b", "maxSpeed_b_long", "maxSpin_b_long", 
              "totalDistance_p", "totalDistance_p_long", "launchSpeed_p", "launchSpeed_p_long",
              "ab_b", "pa_b", "ab_b_long", "pa_b_long", 
              "ab_p", "pa_p", "ab_p_long", "pa_p_long"]

### batter_inputs
batter_inputs = batter_stats_short + batter_stats_long
batter_inputs = [item for item in batter_inputs if item not in pa_exclude]

### pitcher_inputs
pitcher_inputs = pitcher_stats_short + pitcher_stats_long
pitcher_inputs = [item for item in pitcher_inputs if item not in pa_exclude]

# Hand-specific stats (not in PA model)
batter_stats_l = [f'{stat}_l' for stat in batter_inputs]
batter_stats_r = [f'{stat}_r' for stat in batter_inputs]

pitcher_stats_l = [f'{stat}_l' for stat in pitcher_inputs]
pitcher_stats_r = [f'{stat}_r' for stat in pitcher_inputs]

# Hand dummies
hand_inputs = ['p_L', 'b_L']

# Venue ID numbers
venue_nums = ['1', '2', '3', '4', '5', '7', '10', '12', '13', '14', '15', '16', '17', '19', '22', '31', '32', 
              '680', '2392', '2394', '2395', '2535', '2536', '2602', '2680', '2681', '2701', '2735', '2756', 
              '2889', '3289', '3309', '3312', '3313', '4169', '4705', '5010', '5325', '5365', '5381', '5445']

# Venue dummies
venue_inputs = [f"venue_{num}" for num in venue_nums]

# Year dummies
year_inputs = [f"year_{year}" for year in range(2015,2025)]

# Weather inputs
weather_inputs = ['x_vect','y_vect','temperature']

# Game state inputs
game_state_inputs = ['onFirst','onSecond','onThird','top','outs_pre','score_diff']

# Imputation flag dummies
imp_inputs = ['imp_b', 'imp_p']

# Starter dummy
starter_inputs = ['starter']

# All inputs into final PA model
pa_inputs = batter_inputs + pitcher_inputs + hand_inputs + venue_inputs + year_inputs + weather_inputs + game_state_inputs + imp_inputs + starter_inputs

### Pull Inputs

In [ ]:
# List of rolling stats in pull inputs (these are derived from other status and therefore shouldn't be included in models_inputs_plus)
rolled_sums_list = [f'{stat}_sum' for stat in events_list] + ['rbi_sum', 'faced_sum']

# Pull inputs related to the game state
# OUT may be redundant, but it still might help
pull_inputs_game = ['OUT','pitcherScore','batterScore','inning_adj','outs_adj','faced_inning','br_inning','onFirst','onSecond','onThird']
pull_inputs_other = ['IP_start'] # + year_inputs

# All inputs into final pulls model (specifically, from the API/Statcast, not Steamer)
pull_inputs_pre = rolled_sums_list + pull_inputs_game + pull_inputs_other
# Elements to remove
pull_exclude = ["fo_sum", "go_sum", "lo_sum", "po_sum", "rbi_sum"] # Definitely do not want RBI sum as it's redundant with batterScore (we have no unearned runs)

# Create a new list with elements not in the elements_to_remove list
pull_inputs = [element for element in pull_inputs_pre if element not in pull_exclude]